In [ ]:
# Plots for section 5.2 hydrogen technologies

### Imports

In [ ]:
import pandas as pd
import matplotlib
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
from datetime import datetime
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
import seaborn as sns
from matplotlib.lines import Line2D
import math
import matplotlib.ticker as mtick


# imported own functions
from utils import nodal_balance

# imported own definitions
from utils import carrier_colors, carrier_renaming, resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal, convert_ISO_3166_2_to_1
from utils import c_h2_gen, c_h2_con, c_el_gen_s, c_el_con_s

# general variables
font1 = {'fontname':'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/5.2_hydrogen_technologies'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

In [ ]:
# Network imports
stst =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp =pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

In [ ]:
# stst and exp dataframes
# spatial
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

# temporal
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [ ]:
# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger", "solar thermal"]

In [ ]:
# Notebook Functions


In [ ]:
# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

### General

In [ ]:
# efficiencies
stst.links[stst.links.carrier == "H2 Electrolysis"].efficiency.unique()  # Electrolysis: 68 % efficiency
stst.links[stst.links.carrier == "H2 Fuel Cell"].efficiency.unique()  # Fuel Cell: 50 % efficiency
stst.links[stst.links.carrier == "battery charger"].efficiency.unique()  # Fuel Cell: 0.979796 efficiency
stst.links[stst.links.carrier == "battery discharger"].efficiency.unique()  # Fuel Cell: 0.979796 efficiency
# storage losses?

![](../../../../Pictures/Screenshots/Screenshot_20230219_110123.png)
Colbertado

## Interaction of technologies

### Hydrogen Balances

**Differences:**
- stst has no 'H2 pipeline', 'H2 pipeline retrofitted'
- exp has no 'H2 Fuel Cell', 'SMR', 'SMR CC'

In [ ]:
carrier_colors = {
    'AC': 'dimgrey',
    'BEV charger': 'lightyellow',
    'DAC': 'darkgrey',
    'DC': 'lightgrey',
    'Fischer-Tropsch': 'gold',
    'H2': 'turquoise',
    'H2 storage': 'turquoise',
    'H2 Electrolysis': 'fuchsia',
    'H2 Fuel Cell': 'indigo',
    'H2 for industry': 'cadetblue',
    'H2 for shipping': 'navy',
    'H2 liquefaction': 'lightskyblue',
    'Li ion': 'greenyellow',
    'low-temperature heat for industry': "lightsalmon",
    'OCGT': 'saddlebrown',
    'PHS': 'dodgerblue',
    'SMR': 'darkseagreen',
    'SMR CC': 'palegreen',
    'Sabatier': 'peachpuff',
    'V2G': 'honeydew',
    'agriculture electricity': 'goldenrod',
    'agriculture heat': 'peru',
    'battery': 'hotpink',
    'battery charger': 'pink',
    'battery discharger': 'thistle',
    'electricity': 'palegoldenrod',
    'electricity distribution grid': 'thistle',
    'gas': 'orange',
    'home battery': 'violet',
    'home battery charger': 'blueviolet',
    'home battery discharger': 'plum',
    'hydro': 'cornflowerblue',
    'industry electricity': 'rosybrown',
    'land transport EV': 'chocolate',
    'land transport fuel cell': 'peru',
    'offwind-ac': 'cyan',
    'offwind-dc': 'maroon',
    'oil': 'k',
    'onwind': 'green',
    'residential rural gas boiler': 'gold',
    'residential rural ground heat pump': 'darkred',
    'residential rural heat': 'rosybrown',
    'residential rural resistive heater': 'lightgreen',
    'residential rural water tanks charger': 'cadetblue',
    'residential rural water tanks discharger': 'palevioletred',
    'residential urban decentral air heat pump': 'firebrick',
    'residential urban decentral gas boiler': 'lemonchiffon',
    'residential urban decentral heat': 'lightcoral',
    'residential urban decentral resistive heater': 'lightseagreen',
    'residential urban decentral solar thermal': 'salmon',
    'residential urban decentral water tanks charger': 'powderblue',
    'residential urban decentral water tanks discharger': 'crimson',
    'ror': 'blue',
    'services rural gas boiler': 'khaki',
    'services rural ground heat pump': 'maroon',
    'services rural heat': 'indianred',
    'services rural resistive heater': 'seagreen',
    'services rural water tanks charger': 'lightblue',
    'services rural water tanks discharger': 'pink',
    'services urban decentral air heat pump': 'orangered',
    'services urban decentral gas boiler': 'palegoldenrod',
    'services urban decentral heat': 'brown',
    'services urban decentral resistive heater': 'mediumspringgreen',
    'services urban decentral solar thermal': 'tomato',
    'services urban decentral water tanks charger': 'deepskyblue',
    'services urban decentral water tanks discharger': 'lightpink',
    'solar': 'yellow',
    'solar rooftop': 'brown',
    'urban central air heat pump': 'red',
    'urban central gas CHP': 'darkorange',
    'gas CHP': 'darkorange',
    'urban central gas CHP CC': 'navajowhite',
    'gas CHP CC': 'navajowhite',
    'urban central gas boiler': 'darkkhaki',
    'urban central heat': 'firebrick',
    'urban central resistive heater': 'lime',
    'urban central solar thermal': 'pink',
    'urban central solid biomass CHP CC': 'navajowhite',
    'biomass CHP CC': 'navajowhite',
    'urban central solid biomass CHP': 'sandybrown',
    'biomass CHP': 'sandybrown',
    'urban central water tanks charger': 'skyblue',
    'urban central water tanks discharger': 'mediumvioletred',
    'resistive heater': 'seagreen',
    'gas boiler': 'khaki',
    'heat pump': 'darkred',
    'water tanks charger': 'cadetblue',
    'water tanks discharger': 'palevioletred',
    'solar thermal': 'tomato',
    'H2 pipeline': 'pink',
    'H2 pipeline retrofitted': 'violet',
    'naphtha for industry': 'chartreuse',
    'kerosene for aviation': 'skyblue',
    'agriculture machinery oil': 'tan',
    'biogas to gas': "yellowgreen",
    'gas for industry': "olive",
    'gas for industry CC': "mediumslateblue"
}

#### all year

In [ ]:
model = "STST"
carrier = ["H2"]
loads = ["land transport fuel cell", "H2 for industry", "H2 for shipping"]
loads_not_liquid = ["land transport fuel cell", "H2 for industry"]
period = "2013"
# exclude all technologies that contribute less than th
th = 0.005

# resolution
# "8h", "D", "W", "M"
res = "D"

if model == "STST":
    n = stst
elif model == "EXP":
    n = exp

nb_h2 = nodal_balance(n, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_h2 = nb_h2.unstack(level=[1]) / 1e3
# calc loads
loads_h2 = abs(nb_h2[loads_not_liquid].sum(axis=1))
# drop loads
#nb_h2.drop(loads_not_liquid, axis=1, inplace=True)
# rename unhandy column names
nb_h2.rename(columns=carrier_renaming, inplace=True)

####### plot #######

# all year

fig, ax = plt.subplots(figsize=(14, 8))

# resample
df = nb_h2.resample(res).sum()
df_loads = loads_h2.resample(res).sum()

# rename H2 as H2 storage
df.rename(columns={"H2":"H2 storage"}, inplace=True)

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)

df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period].resample(res).mean()
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses)", secondary_y=True)
ax2.set_ylabel("lmp [$€/MWh_{h2}$]")
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="direct hydrogen consumption\n(without H2 for shipping)")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("total hydrogen balance [$GW_{h2}$]")
ax.set_xlabel("")
ax.set_title(f"Hydrogen Balance for {period} ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_bal_all_year_{model}.png")

In [ ]:
# "2013-04-30":"2013-09-30"
(df_pos.resample("m").sum() / 1e3).loc["2013-04-30":"2013-09-30"].mean()
#(df_pos.resample("m").sum() / 1e3).loc[["2013-01-31","2013-02-28", "2013-03-31", "2013-10-31", "2013-11-30","2013-12-31"]].mean()

In [ ]:
nb_h2

In [ ]:
df_pos["H2 Electrolysis"].corr(lmps)

In [ ]:
lmps.describe()

In [ ]:
nb_h2.resample("D").sum()

In [ ]:
# per hour
exp.links_t.p0.loc[: , exp.links.carrier == "H2 liquefaction"].sum(axis=1)/ 1e3 #* 3

In [ ]:
(stst.loads_t.p.loc[: , stst.loads.carrier == "H2 for shipping"].sum(axis=1) / 1e3 * 3)

In [ ]:
(df_neg / 1e3)#.describe()

In [ ]:
df_neg["Fischer-Tropsch"].corr(df_pos.sum(axis=1))

In [ ]:
df_pos["H2 Electrolysis"].corr(df_stst_ts["solar_gen_el"])

In [ ]:
df_stst_ts["onwind_gen_el"]+df_stst_ts["solar_gen_el"]

#### January

In [ ]:
model = "STST"
carrier = ["H2"]
loads = ["land transport fuel cell", "H2 for industry", "H2 for shipping"]
loads_not_liquid = ["land transport fuel cell", "H2 for industry"]
period = "2013-01"
# exclude all technologies that contribute less than th
th = 0.005

# resolution
# "8h", "D", "W", "M"
res = "3h"

if model == "STST":
    n = stst
elif model == "EXP":
    n = exp

nb_h2 = nodal_balance(n, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_h2 = nb_h2.unstack(level=[1]) / 1e3
# calc loads
loads_h2 = abs(nb_h2[loads_not_liquid].sum(axis=1))
# drop loads
nb_h2.drop(loads_not_liquid, axis=1, inplace=True)
# rename unhandy column names
nb_h2.rename(columns=carrier_renaming, inplace=True)

####### plot #######

# all year

fig, ax = plt.subplots(figsize=(14, 8))

# resample
df = nb_h2.resample(res).sum()
df_loads = loads_h2.resample(res).sum()

# drop H2
df.drop("H2", axis=1, inplace=True)

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)

df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses) [$€/MWh_{h2}$]", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="hydrogen loads\n(without H2 for shipping)")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("Total hydrogen balance [$GWh_{h2}$]")
ax.set_xlabel("")
ax.set_title(f"Hydrogen Balance for {period} ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_bal_january_{model}.png")

#### May

In [ ]:
model = "EXP"
carrier = ["H2"]
loads = ["land transport fuel cell", "H2 for industry", "H2 for shipping"]
loads_not_liquid = ["land transport fuel cell", "H2 for industry"]
period = "2013-05"
# exclude all technologies that contribute less than th
th = 0.005

# resolution
# "8h", "D", "W", "M"
res = "3h"

if model == "STST":
    n = stst
elif model == "EXP":
    n = exp

nb_h2 = nodal_balance(n, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_h2 = nb_h2.unstack(level=[1]) / 1e3
# calc loads
loads_h2 = abs(nb_h2[loads_not_liquid].sum(axis=1))
# drop loads
nb_h2.drop(loads_not_liquid, axis=1, inplace=True)
# rename unhandy column names
nb_h2.rename(columns=carrier_renaming, inplace=True)

####### plot #######

# all year

fig, ax = plt.subplots(figsize=(14, 8))

# resample
df = nb_h2.resample(res).sum()
df_loads = loads_h2.resample(res).sum()

# drop H2
df.drop("H2", axis=1, inplace=True)

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)

df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses) [$€/MWh_{h2}$]", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="hydrogen loads\n(without H2 for shipping)")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("Total hydrogen balance [$GWh_{h2}$]")
ax.set_xlabel("")
ax.set_title(f"Hydrogen Balance for {period} ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_bal_may_{model}.png")

## Focus on technologies

### Duration curves

In [ ]:
df_stst_ts[[c + "_gen_h2" for c in c_h2_gen]]

In [ ]:
df_exp_ts[[c + "_gen_h2" for c in c_h2_gen]]
df_con = df_stst_ts[[c + "_con_h2" for c in c_h2_con]]

In [ ]:
[c + "_con_h2" for c in c_h2_con]

In [ ]:
# load duration curves of hydrogen producing and consuming technologies
# IMPORTANT: production, consumption and loads have to be divided by 3. If you do not do it every time sample is in total energy and this means for 3 hours

model = "STST"

if model == "STST":
    n = stst
    df = df_stst_ts
    df_gen = df_stst_ts[[c + "_gen_h2" for c in c_h2_gen]]
    df_con = df_stst_ts[[c + "_con_h2" for c in c_h2_con]]
elif model == "EXP":
    n = exp
    df = df_exp_ts
    df_gen = df_exp_ts[['H2 Electrolysis_gen_h2']]
    df_con = df_exp_ts[['H2 liquefaction_con_h2', 'Sabatier_con_h2', 'Fischer-Tropsch_con_h2']]

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(16, 7))

for i, c in enumerate(df_gen.columns):
    gen = df_gen[c] / 1e3 / 3
    gen = pd.DataFrame(gen.sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c.split('_')[0]], label=c.split('_')[0])

# plot load
# just not multiplying with 3 to obtain value in (MWh)
load = (n.loads_t.p.loc[: , n.loads.carrier.isin(loads_not_liquid)].sum(axis=1) / 1000)
ax1.hlines(y=load.mean(), xmin=0, xmax=1, ls="dashed", linewidth=1, color='saddlebrown', label="hydrogen load (mean)")
ax1.hlines(y=load.max(), xmin=0, xmax=1, ls="dashed", linewidth=1, color='red', label = "hydrogen load (max)")

ax1.set_ylabel("production [$GW_{h2}$]")
ax1.set_xlabel("fraction of total time")
ax1.set_facecolor("lightgrey")
ax1.legend()
ax1.grid(True)
ax1.set_title(f"Duration curves of hydrogen producing technologies ({model})", fontsize=16, **font1)

for i, c in enumerate(df_con.columns):
    con = df_con[c] / 1e3 / 3
    con = pd.DataFrame(con.sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(con, color = carrier_colors[c.split('_')[0]], label=c.split('_')[0])

ax2.set_ylabel("production [$GW_{h2}$]")
ax2.set_xlabel("fraction of total time")
ax2.set_facecolor("lightgrey")
ax2.legend(ncol=2)
ax2.grid(True)
ax2.set_title(f"Duration curves of hydrogen consuming technologies ({model})", fontsize=16, **font1)

fig.tight_layout()
#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_dur_{model}.png")

In [ ]:
# comparison
gen = df_stst_ts[['H2 Electrolysis_gen_h2']] / 1e3
gen = pd.DataFrame(gen.sort_values(ascending=False, by='H2 Electrolysis_gen_h2')).set_index(pd.Index(np.linspace(0, 1, num=2920)))
plt.plot(gen, label="stst")

gen = df_exp_ts[['H2 Electrolysis_gen_h2']] / 1e3
gen = pd.DataFrame(gen.sort_values(ascending=False, by='H2 Electrolysis_gen_h2')).set_index(pd.Index(np.linspace(0, 1, num=2920)))
plt.plot(gen, label="exp")
plt.legend()

In [ ]:
stst.links[stst.links.carrier == "SMR CC"]

In [ ]:
np.multiply(stst.links.p_nom_opt[stst.links.carrier == "SMR CC"], stst.links.efficiency[stst.links.carrier == "SMR CC"]).sum() / 1e3

In [ ]:
df_stst_ts["SMR CC_gen_h2"].sort_values() / 1e3 / 3

In [ ]:
# new indices: cut off Fuel cell and SMR completely and SMR CC for exp
c_h2_gen_stst = ['H2 Electrolysis', 'SMR CC']
c_h2_gen_exp = ['H2 Electrolysis']
c_h2_gen = ['H2 Electrolysis', 'SMR CC']
c_h2_con = ['H2 liquefaction', 'Sabatier', 'Fischer-Tropsch']

### Capacity factors

In [ ]:
# delete cf for SMR CC in exp
df_exp_ons["SMR CC_cf_gen_h2"] = pd.DataFrame(np.nan, index=df_exp_ons.index, columns = ["SMR CC_cf_gen_h2"])

##### Boxplot

In [ ]:
# capacity factors across regions as boxplot (STST & EXP)

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
c_h2_con_index = [x for x in c_h2_con if x != 'H2 liquefaction']
stst_h2_gen = df_stst_ons[[c+ "_cf_gen_h2" for c in c_h2_gen] + [c + "_cf_con_h2" for c in c_h2_con_index]]
stst_h2_gen = stst_h2_gen.values
exp_h2_gen = df_exp_ons[[c+ "_cf_gen_h2" for c in c_h2_gen]+ [c + "_cf_con_h2" for c in c_h2_con_index]]
exp_h2_gen = exp_h2_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_h2_gen)
filtered_stst = [d[m] for d, m in zip(stst_h2_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_h2_gen)
filtered_exp = [d[m] for d, m in zip(exp_h2_gen.T, mask_exp.T)]

cf_index = [c+ "_cf_gen_h2" for c in c_h2_gen] + [c + "_cf_con_h2" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen] + [c + "_con_h2" for c in c_h2_con_index]
index = c_h2_gen + c_h2_con_index
ticks = [carrier_renaming.get(n, n) for n in index]
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops,
                        zorder=1
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops,
                       zorder=2
                       )

# generation weighted mean
gwm_cf_stst =np.multiply(df_stst_ons[cf_index], (df_stst_ons[gen_con_index] / df_stst_ons[gen_con_index].sum())).sum()
gwm_cf_exp =np.multiply(df_exp_ons[cf_index], (df_exp_ons[gen_con_index] / df_exp_ons[gen_con_index].sum())).sum()
# set value for SMR CC in exp to nan
gwm_cf_exp["SMR CC_cf_gen_h2"] = np.nan

ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, gwm_cf_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, gwm_cf_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[cf_index].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1.01), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[cf_index].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.xticks(rotation=0)
# plt.title("Market values of electricity producing technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)
ax.grid(True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_ylabel("capacity factor [%]")

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
ax.legend([patch1, patch2], ['STST', 'EXP'], loc="lower right")

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_gen_con_cap_fac_STST_EXP.png")

In [ ]:
df_stst = df_stst_ons[[c+ "_cf_gen_h2" for c in c_h2_gen] + [c + "_cf_con_h2" for c in c_h2_con_index]]
df_exp = df_exp_ons[[c+ "_cf_gen_h2" for c in c_h2_gen] + [c + "_cf_con_h2" for c in c_h2_con_index]]

In [ ]:
df_stst.describe()

In [ ]:
df_exp.describe()

In [ ]:
df_exp_ons[["Sabatier_cf_con_h2", "Sabatier_con_h2"]].sort_values(by="Sabatier_con_h2",ascending=True)

In [ ]:
df_stst_ons[gen_con_index]

In [ ]:
gwm_cf_stst

In [ ]:
gwm_cf_exp

In [ ]:
# wtf is going wrong here?
# df_stst_ons["H2 liquefaction_cf_con_h2"] = np.ones(181)

In [ ]:
df_stst_ons["H2 liquefaction_cf_con_h2"] = np.ones(181)

In [ ]:
cf_index

##### Map

In [ ]:
# capacity factors across regions as map

model = "STST"

if model == "STST":
    df = df_stst_ons
    c_h2_gen_index = c_h2_gen_stst

elif model == "EXP":
    df = df_exp_ons
    c_h2_gen_index = c_h2_gen_exp

carriers = c_h2_gen + c_h2_con
col_names = [[c+ "_cf_gen_h2" for c in c_h2_gen_index ] + [c + "_cf_con_h2" for c in c_h2_con]]

cf_index = [c+ "_cf_gen_h2" for c in c_h2_gen_index] + [c + "_cf_con_h2" for c in c_h2_con]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con]
index = c_h2_gen_index + c_h2_con
ticks = [carrier_renaming.get(n, n) for n in index]

fig, axs = plt.subplots(ncols=2, nrows=math.ceil(len(index)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, math.ceil(len(index)/2) * 6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{cf_index[i]}",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"capacity factors",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[gen_con_index[i]].abs().max()
    # blue if negative and green if positive
    colors = ['darkblue' if (x < 0) else 'darkgreen' for x in df[gen_con_index[i]] ]

    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(df[gen_con_index[i]].abs() / max_size) * 300,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="darkgreen", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="darkblue", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Production', 'Consumption', f"max circle size:\n{round(max_size)} {unit}"), numpoints=1, loc="upper left", prop={'size': 9})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)


    ax.set_title(f"{ticks[i]}", fontsize=16, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout()
plt.show()

In [ ]:
# capacity factors across regions as map: liquefaction at end

model = "STST"

if model == "STST":
    df = df_stst_ons
    c_h2_gen_index = c_h2_gen_stst
    cf_index = ['H2 Electrolysis_cf_gen_h2', 'SMR CC_cf_gen_h2', 'Sabatier_cf_con_h2', 'Fischer-Tropsch_cf_con_h2', 'H2 liquefaction_cf_con_h2',]
    gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + ['Sabatier_con_h2', 'Fischer-Tropsch_con_h2','H2 liquefaction_con_h2']
    index = ['H2 Electrolysis', 'SMR CC', 'Sabatier', 'Fischer-Tropsch', 'H2 liquefaction']

elif model == "EXP":
    df = df_exp_ons
    c_h2_gen_index = c_h2_gen_exp
    cf_index = ['H2 Electrolysis_cf_gen_h2', 'Sabatier_cf_con_h2', 'Fischer-Tropsch_cf_con_h2', 'H2 liquefaction_cf_con_h2',]
    gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + ['Sabatier_con_h2', 'Fischer-Tropsch_con_h2','H2 liquefaction_con_h2']
    index = ['H2 Electrolysis', 'Sabatier', 'Fischer-Tropsch', 'H2 liquefaction']

ticks = [carrier_renaming.get(n, n) for n in index]

fig, axs = plt.subplots(ncols=2, nrows=math.ceil(len(index)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, math.ceil(len(index)/2) * 6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    if ticks[i] != "H2 liquefaction":
        df.to_crs(crs.proj4_init).plot(column=f"{cf_index[i]}",
                                       ax=ax,
                                       cmap=plt.get_cmap("magma_r"),
                                       linewidth=0.05,
                                       edgecolor = 'grey',
                                       legend= True,
                                       legend_kwds={'label':"capacity factors",'orientation': "vertical",'shrink' : 0.9}
                                       )

    max_size = df[gen_con_index[i]].abs().max()
    # blue if negative and green if positive
    colors = ['darkblue' if (x < 0) else 'darkgreen' for x in df[gen_con_index[i]] ]

    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(df[gen_con_index[i]].abs() / max_size) * 300,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="darkgreen", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="darkblue", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Production', 'Consumption', f"max circle size:\n{round(max_size)} {unit}"), numpoints=1, loc="upper left", prop={'size': 9})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)


    ax.set_title(f"{ticks[i]}", fontsize=16, **font1)

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/cap_fac_h2_gen_con_{model}.png")

In [ ]:
df_exp_ons[["H2 Electrolysis_cf_gen_h2", "H2 Electrolysis_gen_h2"]].sort_values(by = "H2 Electrolysis_gen_h2", ascending=False)

In [ ]:
n = stst
loads_h2 = (n.loads_t.p.loc[: , n.loads.carrier.isin(loads)] / 1e3 * 3).sum()
loads_h2.index = loads_h2.index.map(n.loads.bus).map(stst.buses.location)
loads_h2 = loads_h2.groupby(loads_h2.index, axis=0).sum()

In [ ]:
# correlation of loads and production
df_stst_ons["SMR CC_gen_h2"].corr(loads_h2)

# STST+electrolysis: 0.19
# STST+SMR CC: 0.78
# STST+electrolysis: 0.08

In [ ]:
df_stst_ons["Sabatier_con_h2"].abs().sum()  / df_exp_ons["Sabatier_con_h2"].abs().sum()

In [ ]:
i_bi = df_stst_ons.index[df_stst_ons.index.str.contains("GB")].tolist() + df_stst_ons.index[df_stst_ons.index.str.contains("IE")].tolist()

In [ ]:
df_stst_ons.loc[i_bi , "Sabatier_con_h2"].abs().sum()  / df_stst_ons["Sabatier_con_h2"].abs().sum() # 0.47
df_exp_ons.loc[i_bi , "Sabatier_con_h2"].abs().sum()  / df_exp_ons["Sabatier_con_h2"].abs().sum() # 100

In [ ]:
df_stst_ons.loc[i_bi , "Sabatier_cf_con_h2"].mean()
df_exp_ons.loc[i_bi , "Sabatier_cf_con_h2"].mean()

In [ ]:

df_stst_ons.loc["AL0 0", "test"] = 5
#df_stst_ons["H2 liquefaction_cf_con_h2"]

In [ ]:
df_exp_ons["Fischer-Tropsch_con_h2"].abs().sum()  / 1e3 #df_exp_ons["Fischer-Tropsch_con_h2"].abs().sum()

In [ ]:
df_exp_ons["Fischer-Tropsch_con_h2"].corr(df_exp_ons["Fischer-Tropsch_cf_con_h2"])
#df_stst_ons["Fischer-Tropsch_con_h2"].corr(df_stst_ons["Fischer-Tropsch_cf_con_h2"])

In [ ]:
np.ones(181)

In [ ]:
# why is it plotting wrong values?
df_stst_ons["test"] = np.ones(181)

crs = ccrs.EqualEarth()

df_stst_ons.to_crs(crs.proj4_init).plot(column="test",
                               cmap=plt.get_cmap("magma_r"),
                               legend=True
                               )

In [ ]:
df_stst_ons["H2 liquefaction_cf_con_h2"]

In [ ]:
# why is it plotting wrong values?

fig, ax = plt.subplots(ncols=1, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, 6))
crs = ccrs.EqualEarth()
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')

df_stst_ons.to_crs(crs.proj4_init).plot(column=f"H2 liquefaction_cf_con_h2",
                               ax=ax,
                               cmap=plt.get_cmap("magma_r"),
                               linewidth=0.05,
                               edgecolor = 'grey',
                               legend=True,
                               legend_kwds={'label':"capacity factors",'orientation': "vertical",'shrink' : 0.9}
                               )

#### Reasons for the difference in capacity factors

In [ ]:
# share of electricity generation from renewables per region
df = df_exp_ons# df_stst_ons

vre = ['onwind', 'solar', 'offwind-dc', 'solar rooftop', 'offwind-ac', 'ror']
onwind_solar = ['onwind', 'solar']
all_gen_el = c_el_gen_s

df["share_vre_gen_el"] = df[[c + "_gen_el" for c in vre]].sum(axis=1) / df[[c + "_gen_el" for c in c_el_gen_s]].sum(axis=1)
df["share_onwind+solar_gen_el"] = df[[c + "_gen_el" for c in onwind_solar]].sum(axis=1) / df[[c + "_gen_el" for c in c_el_gen_s]].sum(axis=1)

In [ ]:
countries = ["ES", "PT"]

i = []
for country in countries:
    i += df.index[df.index.str.contains(country)].tolist()

i

In [ ]:
df.loc[i,"H2 Electrolysis_cf_gen_h2"].mean()

In [ ]:
df["H2 Electrolysis_cf_gen_h2"].corr(df[[c + "_gen_el" for c in vre]].sum(axis=1))
#df["H2 Electrolysis_cf_gen_h2"].corr(df["share_vre_gen_el"])
#df["H2 Electrolysis_cf_gen_h2"].corr(df["share_onwind+solar_gen_el"])

In [ ]:
df.loc[i,"H2 Electrolysis_cf_gen_h2"].corr(df.loc[i,"share_vre_gen_el"])

In [ ]:
df_stst_ons[[c + "_gen_el" for c in vre]].sum(axis=1)

In [ ]:
# plot of production weighted capacity factors by country
df_stst_ons_help = df_stst_ons.copy()
df_stst_ons_help["country_help"] = df_stst_ons_help.index.str[:2]
ccs = [convert_ISO_3166_2_to_1[cc] for cc in df_stst_ons_help["country_help"]]
df_stst_ons_help["country"] = pd.DataFrame([ccs]).T.values

df_stst_ons_c = pd.DataFrame(index = df_stst_ons_help["country"])
df_stst_ons_c["H2 Electrolysis_cf*gen_h2"] = pd.concat([np.multiply(df_stst_ons_help["H2 Electrolysis_cf_gen_h2"], df_stst_ons_help["H2 Electrolysis_gen_h2"]) , df_stst_ons_help["country"]], axis=1).groupby(by="country").sum()
df_stst_ons_c["H2 Electrolysis_gen_h2"] = pd.concat([df_stst_ons_help["H2 Electrolysis_gen_h2"] , df_stst_ons_help["country"]], axis=1).groupby(by="country").sum()
df_stst_ons_c["H2 Electrolysis_cf_gen_h2"] = df_stst_ons_c["H2 Electrolysis_cf*gen_h2"] / df_stst_ons_c["H2 Electrolysis_gen_h2"]

df_stst_ons_c

In [ ]:
fig, ax = plt.subplots(figsize=(16, 5))

ax.plot(df_stst_ons_c["H2 Electrolysis_cf_gen_h2"])

In [ ]:
df_stst_ons_help["country"]

In [ ]:
df_exp_ts["Fischer-Tropsch_con_h2"].corr(exp.buses_t.marginal_price.loc[: , exp.buses.carrier == "H2"].sum(axis=1))

In [ ]:
df_stst_ons["Sabatier_con_h2"].corr(df_stst_ons["urban central gas CHP_gen_heat"])

In [ ]:
df_exp_ons["Sabatier_con_h2"].corr(df_exp_ons["urban central gas CHP_gen_heat"])

### production and capacity difference

In [ ]:
c_h2_gen_index = c_h2_gen
# remove H2 liquefaction as the capacity and the consumption is exactly the same in both scenarios
c_h2_con_index = [x for x in c_h2_con if x != 'H2 liquefaction']

cap_index = [c+ "_cap_gen_h2" for c in c_h2_gen_index] + [c + "_cap_con_h2" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con_index]
index = c_h2_gen_index + c_h2_con_index
ticks = [carrier_renaming.get(n, n) for n in index]

# STST - EXP
for c in cap_index:
    df_stst_ons[f"{c}_STST-EXP"] = df_stst_ons[c] - df_exp_ons[c]

fig, axs = plt.subplots(ncols=2, nrows=math.ceil(len(index)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, math.ceil(len(index)/2) * 6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    abs_max = df_stst_ons[f"{cap_index[i]}_STST-EXP"].abs().max()
    #unit_cap = "TW" if abs_max > 1e4 else "GW"
    #df[f"{carriers[i]}_cap_STST-EXP"] = (df[f"{carriers[i]}_cap_STST-EXP"] / 1e3) if abs_max > 1e4 else df[f"{carriers[i]}_cap_STST-EXP"]
    #abs_max = abs_max / 1e3 if abs_max > 1e4 else abs_max

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df_stst_ons.to_crs(crs.proj4_init).plot(column=f"{cap_index[i]}_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"Capacity differences ($GW_{h2}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df_stst_ons[gen_con_index[i]].abs() - df_exp_ons[gen_con_index[i]].abs()
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df_stst_ons.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased production / consumption in STST', 'Increased production / consumption in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{ticks[i]} capacity (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the capacity and production for electricity producing VRE technologies (STST -EXP)", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_cap_STST-EXP_map_all.png")

In [ ]:
df_stst_ts["H2 liquefaction_gen_h2"]

In [ ]:
i = 2
df_stst_ons[f"{cap_index[i]}_STST-EXP"].sort_values(ascending=False)

In [ ]:
gen_diff = df_stst_ons[gen_con_index[i]].abs() - df_exp_ons[gen_con_index[i]].abs()
gen_diff.sort_values(ascending=False) / 1e3

### production heatmap

In [ ]:
# production heatmap and average

model = "EXP"

if model == "STST":
    n = stst
    df_1 = df_stst_ts
    c_h2_gen_index = c_h2_gen_stst
elif model == "EXP":
    n = exp
    df_1 = df_exp_ts
    c_h2_gen_index = c_h2_gen_exp

c_h2_con_index = [x for x in c_h2_con if x != 'H2 liquefaction']

cap_index = [c+ "_cap_gen_h2" for c in c_h2_gen_index] + [c + "_cap_con_h2" for c in c_h2_con_index]
gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con_index]
index = c_h2_gen_index + c_h2_con_index
ticks = [carrier_renaming.get(n, n) for n in index]


fig, axs = plt.subplots(nrows=math.ceil(len(index)/2), ncols=2, figsize=(18, math.ceil(len(index)/2)*5))

for i, ax in enumerate(axs.reshape(-1)):

    if i >= len(index):
        ax.axis('off')
        continue

    # production in TWh
    df = df_1[gen_con_index[i]] / 1e3

    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax=ax,
                cmap=(plt.get_cmap("magma_r") if ticks[i] in c_h2_gen else plt.get_cmap("magma")),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': f"{'production' if ticks[i] in c_h2_gen else 'consumption'} in GWh", 'pad': 0.1})

    ax.set_title(f"{ticks[i]}", fontsize=16, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

    # Plot production profile on second y-axis
    # resample and transform axis of date to index (heatmap works not with dates)
    df2 = df.resample("5D").sum() / 1e3
    df2 = df2[df2.index.year == 2013]
    df2.index = df2.index.dayofyear - 0.5

    ax2 = ax.twinx()
    ax2.plot(df2, color="white", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    ax2.legend( loc='lower right', labels=[f"{'production' if ticks[i] in c_h2_gen else 'consumption'} (right axis)"])
    ax2.set_ylabel(f"{'production' if ticks[i] in c_h2_gen else 'consumption'} in TWh (5-day sum)", fontsize=12, **font1)
    ax2.grid(False)

fig.tight_layout(pad=3)
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_gen_con_profiles_{model}.png")

In [ ]:
df_stst_ts["H2 Electrolysis_gen_h2"].corr(df_stst_ts["SMR CC_gen_h2"])

### Storage

In [ ]:
# H2 stores
c = "H2"
stst.stores[stst.stores.carrier == c]

In [ ]:
h2_cap = stst.stores.e_nom_opt[stst.stores.carrier == c]
h2_cap.index = h2_cap.index.map(n.stores.bus).map(stst.buses.location)
h2_cap

In [ ]:
# storage capacity (TWh)
c = "H2"

for n, df in zip([stst, exp], [df_stst_ons, df_exp_ons]):
    h2_cap = n.stores.e_nom_opt[n.stores.carrier == c]
    h2_cap.index = h2_cap.index.map(n.stores.bus).map(n.buses.location)
    df["h2_store_cap"] = h2_cap / 1e6

# STST: 32.22 TWh (sum)
# EXP: 61.78 TWh (sum)

In [ ]:
# H2 storage capacity map with difference in production and consumption

fig, axs = plt.subplots(ncols=2, nrows=1,
                        subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15,  6))
crs = ccrs.EqualEarth()

# align colorbar for comparison
cap1 = df_stst_ons["h2_store_cap"]
cap2 = df_exp_ons["h2_store_cap"]
vmin = min(np.nanmin(cap1), np.nanmin(cap1))
vmax = max(np.nanmax(cap1), np.nanmax(cap2))

for i, ax in enumerate(axs.reshape(-1)):

    if i == 0:
        model = "STST"
        df = df_stst_ons
        n = stst
        c_h2_gen_index = c_h2_gen_stst
    elif i == 1:
        model = "EXP"
        df = df_exp_ons
        n = exp
        c_h2_gen_index = c_h2_gen_exp

    gen_con_index = [c+ "_gen_h2" for c in c_h2_gen_index] + [c + "_con_h2" for c in c_h2_con]

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column="h2_store_cap",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   vmin=vmin,
                                   vmax=vmax,
                                   legend_kwds={'label':"Capacity ($TWh_{h2}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )
    if model == "EXP":
        # consumption
        loads_nl = (n.loads_t.p.loc[: , n.loads.carrier.isin(loads_not_liquid)] / 1e3 * 3).sum()
        loads_nl.index = loads_nl.index.map(n.loads.bus).map(stst.buses.location)
        loads_nl = loads_nl.groupby(loads_nl.index, axis=0).sum()
        df["gen_con_loads_nl"] = df[gen_con_index].sum(axis=1) - loads_nl


        max_size = df["gen_con_loads_nl"].abs().max()
        # blue if negative and green if positive
        colors = ['darkblue' if (x < 0) else 'darkgreen' for x in df["gen_con_loads_nl"] ]

        df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(df["gen_con_loads_nl"].abs() / max_size) * 300,  color=colors, edgecolor="white")
        circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="darkgreen", markeredgecolor="white", markersize=10)
        circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="darkblue", markeredgecolor="white", markersize=10)
        circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

        unit = "$TWh_{h2}$" if max_size > 1e3 else "$GWh_{h2}$"
        max_size = max_size / 1e3 if max_size > 1e3 else max_size
        ax.legend((circle1, circle2, circle3), ('Excess h2 production', 'Excess h2 consumption', f"max circle size:\n{round(max_size)} {unit}"), numpoints=1, loc="upper left", prop={'size': 9})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)


    ax.set_title(f"Storage capacity H2 ({model})", fontsize=16, **font1)


fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_cap_stor_excess_gen_con_map.png")

In [ ]:
df_exp_ons["h2_store_cap"].sort_values(ascending=False)

In [ ]:
df_stst_ons["h2_store_cap"].sort_values(ascending=False)["GB4 0"]

In [ ]:
# difference of capacities: H2 storage capacity map

fig, ax = plt.subplots(ncols=1, nrows=1,
                        subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14,  6))
crs = ccrs.EqualEarth()

df_stst_ons["h2_store_cap_STST-EXP"] = df_stst_ons["h2_store_cap"] - df_exp_ons["h2_store_cap"]
abs_max = df_stst_ons["h2_store_cap_STST-EXP"].abs().max()

ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')

df_stst_ons.to_crs(crs.proj4_init).plot(column="h2_store_cap_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   linewidth=0.05,
                                   vmin=-abs_max,
                                   vmax=abs_max,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"Capacity ($TW_{h2}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

# always select same section
xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
pad = 1 * 1e5  # add a padding around the geometry
ax.set_xlim(xmin-pad, xmax+pad)
ax.set_ylim(ymin-pad, ymax+pad)

ax.set_title(f"H2 Storage capacity difference (STST-EXP)", fontsize=16, **font1)

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_cap_stor_excess_gen_con_map.png")

In [ ]:
# Why are the hydrogen capacities located there?
# calc Correlation with wind, solar and other power plant capacity (e.g.

In [ ]:
df_stst_ons["h2_store_cap_STST-EXP"].sort_values(ascending=False)

#### Storage Capacity correlation (spatial)

In [ ]:
# correlation with everything in df
df_stst_ons.corr()["h2_store_cap"].sort_values().dropna().tail(50)
df_exp_ons.corr()["h2_store_cap"].sort_values().dropna().tail(50)

**Findings:**

STST+pos: Fischer-Tropsch_cap_con_h2 (0.74), H2 Electrolysis_cap_con_el(0.61)
STST+neg: Fischer-Tropsch_con_h2 (-0.80), H2 Electrolysis_con_el (-0.59),

EXP+pos: Sabatier_cap_con_h2 (0.52), H2 pipeline_cap_gen_h2 (0.48), H2 pipeline_cap (0.4)
EXP+neg: Sabatier_con_h2 (-0.54), H2 pipeline_con_h2 (-0.49),

In [ ]:
index = [c + "_cap_gen_el" for c in c_el_gen_s] + [c + "_cap_con_el" for c in c_el_con_s] + [c + "_cap_gen_h2" for c in  c_h2_gen] + [c + "_cap_con_h2" for c in c_h2_con] + ["h2_store_cap"]
cols = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n) + " (con)" for n in
c_el_con_s] + [carrier_renaming.get(n, n) for n in c_h2_gen] + [carrier_renaming.get(n, n) + " (con)" for n in c_h2_con] + ["h2_store_cap"]

fig, axs = plt.subplots(ncols=2, figsize=(22, 8))

for i, df, ax in zip([0, 1], [df_stst_ons, df_exp_ons], axs):
    corr = df[index].corr()
    corr.index = cols
    corr.columns = cols

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.4], mask=mask, cmap="magma_r", annot=True, ax=ax)
    ax.set_title('STST' if i == 0 else 'EXP')

fig.tight_layout(pad=1.5)
#plt.close()
plt.show()

#### Feed-in and feed-out pattern

In [ ]:
# p: active power at bus (positive if net production) (MW)

# GWh
n = stst
# indices are the same
i_h2_stores = n.stores[n.stores.carrier == "H2"].index

fig, axs = plt.subplots(nrows=2, figsize=(14, 8))

for n, ax in zip([stst, exp], axs):
    (n.stores_t.p[i_h2_stores].sum(axis=1) / 1e3 * 3).resample("3h").sum().plot(ax=ax,
                                                                                lw=0.7,
                                                                                color='turquoise',
                                                                                label="feed- in and feed out pattern")
    (n.stores_t.p[i_h2_stores].sum(axis=1) / 1e3 * 3).resample("D").mean().plot(ax=ax,
                                                                                lw=1,
                                                                                color='black',
                                                                               label="feed- in and feed out pattern (daily mean)")
    ax.set_ylabel("Active power ($GWh_{h2}$)")
    ax.set_xlabel("")
    ax.set_title("STST" if n == stst else "EXP", fontsize=16, **font1)
    ax.legend(loc="lower left")

fig.tight_layout()

#fig.savefig(f"{PLOT_DIR}/h2_stor_feed_pattern.png")

In [ ]:
# correlation between both patterns
stst.stores_t.p[i_h2_stores].sum(axis=1).corr(exp.stores_t.p[i_h2_stores].sum(axis=1))

In [ ]:
# connection between p (MW) and e (MWh)
e = n.stores_t.e[i_h2_stores].sum(axis=1)
p = (n.stores_t.p[i_h2_stores].sum(axis=1) * -1).cumsum() * 3 + n.stores_t.e[i_h2_stores].loc["2013-01-01 00:00:00"].sum()

In [ ]:
plt.plot(e, label="e")
plt.plot(p, label="p")
plt.legend()

In [ ]:
e

In [ ]:
p

In [ ]:
for df, n in zip([df_stst_ts, df_exp_ts], [stst, exp]):
    df["h2_feed_in"] = n.stores_t.p[i_h2_stores][n.stores_t.p[i_h2_stores] >= 0].sum(axis=1) / 1e3 * 3 * -1
    df["h2_feed_out"] = n.stores_t.p[i_h2_stores][n.stores_t.p[i_h2_stores] < 0].sum(axis=1) / 1e3 * 3 * -1
    df["h2_feed_in_out"] = n.stores_t.p[i_h2_stores].sum(axis=1) / 1e3 * 3 * -1
    df["h2_stor_level"] = n.stores_t.e[i_h2_stores].sum(axis=1) / 1e3

#### Storage level over time

In [ ]:
# e: Energy as calculated by the OPF. (MWh)

fig, axs = plt.subplots(nrows=2, figsize=(14, 8))

for n, ax in zip([stst, exp], axs):
    (n.stores_t.e[i_h2_stores].sum(axis=1) / 1e6).resample("3h").sum().plot(ax=ax,
                                                                                lw=0.7,
                                                                                color='turquoise',
                                                                                label="storage level")
    (n.stores_t.e[i_h2_stores].sum(axis=1) / 1e6).resample("D").mean().plot(ax=ax,
                                                                                lw=1,
                                                                                color='black',
                                                                               label="storage level (daily mean)")
    ax.set_ylabel("$TWh_{h2}$")
    ax.set_xlabel("")
    ax.set_title("STST" if n == stst else "EXP", fontsize=16, **font1)
    ax.legend(loc="lower left")

fig.tight_layout()

#fig.savefig(f"{PLOT_DIR}/h2_stor_storage_level.png")

#### lmps and storage


In [ ]:
n = exp

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 4.5))

# change to mean for investigating in mean (unit: €/MWh)
df = pd.DataFrame(n.buses_t.marginal_price[n.buses[n.buses.carrier=="H2"].index].mean(axis=1))

hours = df.index.hour.unique()[::-1]
df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

for hour in hours:
    df_start[str(hour)] = df[df.index.hour==hour].values

sns.heatmap(df_start.transpose(),
            ax =ax,
            cmap=plt.get_cmap("magma_r"),
            linewidth=0.001,
            xticklabels=15,
            cbar_kws={'label': "lmps ($€/MWh_{h2}$)", 'pad': 0.09})
#ax.set_title(f"{carriers[i]}", fontsize=16, **font1)
ax.set_ylabel("hour of the day", fontsize=12, **font1)
ax.set_xlabel("day of the year", fontsize=12, **font1)

# Rewrite the y labels
x_labels = ax.get_xticks()
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

# Plot production profile on second y-axis
# resample and transform axis of date to index (heatmap works not with dates)
df2 = (n.stores_t.e[i_h2_stores].sum(axis=1) / 1e6).resample("D").mean()
df2 = df2 / df2.max()
df2 = df2[df2.index.year == 2013]
df2.index = df2.index.dayofyear - 0.5

# add combined onshore wind and solar production
df3 = df_exp_ts["onwind_gen_el"] + df_exp_ts["solar_gen_el"]
df3 = df3.resample("7D").mean() / df3.resample("7D").mean().max()
df3 = df3[df3.index.year == 2013]
df3.index = df3.index.dayofyear - 0.5


ax2 = ax.twinx()
ax2.plot(df2, color="white", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
ax2.plot(df3, color="green", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
ax2.legend( loc='lower right', labels=["hydrogen storage level (1d mean)", "onwind+solar generation (7d mean)"])
ax2.set_ylabel("storage level / generation in unit of max", fontsize=12, **font1)
ax2.grid(False)

fig.suptitle(f"H2 lmps and hydrogen storage level (EXP)", fontsize=16, **font1)
fig.tight_layout(pad=1)
plt.show()

PLOT_DIR_add = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/additional_stuff'
#fig.savefig(f"{PLOT_DIR_add}/h2_lmps_storage_level_vre_gen_exp.png")

In [ ]:
df_exp_ts["onwind_gen_el"] + df_exp_ts["solar_gen_el"]

In [ ]:
# temporal patterns
c = "H2"
n = exp
d = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)

fig, ax = plt.subplots()
fft = abs(pd.Series(np.fft.rfft(d - d.mean()), index=np.fft.rfftfreq(len(d), d=1./2920))**2)
fft.plot(xlim=[0,1000], ylabel=f"??? Density ({c})", xlabel="Frequency (1/year)")
print(c)
print(pd.DataFrame(fft).sort_values(by=0, ascending=False).head(20))

In [ ]:
# relationship between storage filling level and solar+onwinf generation
# e: Energy as calculated by the OPF. (MWh)

fig, axs = plt.subplots(nrows=2, figsize=(14, 8))

for n, ax in zip([stst, exp], axs):
    (n.stores_t.e[i_h2_stores].sum(axis=1) / 1e6).resample("3h").sum().plot(ax=ax,
                                                                                lw=0.7,
                                                                                color='turquoise',
                                                                                label="storage level")
    (n.stores_t.e[i_h2_stores].sum(axis=1) / 1e6).resample("D").mean().plot(ax=ax,
                                                                                lw=1,
                                                                                color='black',
                                                                               label="storage level (daily mean)")
    sol_win_gen = df_stst_ts[["onwind_gen_el", "solar_gen_el"]].sum(axis=1)
    sol_win_gen = sol_win_gen / max(sol_win_gen) * max((n.stores_t.e[i_h2_stores].sum(axis=1) / 1e6).resample("3h").sum())
    sol_win_gen.resample("5D").mean().plot(ax=ax, label="sol_win_gen", color="red")
    ax.set_ylabel("$TWh_{h2}$")
    ax.set_xlabel("")
    ax.set_title("STST" if n == stst else "EXP", fontsize=16, **font1)
    ax.legend(loc="lower left")
    print(sol_win_gen.corr(n.stores_t.e[i_h2_stores].sum(axis=1)))

fig.tight_layout()

In [ ]:
# relationship between solar + onwind generation and storage filling level not existent?
df_stst_ts[["onwind_gen_el", "solar_gen_el"]].sum(axis=1).corr(stst.stores_t.e[i_h2_stores].sum(axis=1))
df_exp_ts[["onwind_gen_el", "solar_gen_el"]].sum(axis=1).corr(exp.stores_t.e[i_h2_stores].sum(axis=1))

# maybe with feed-in and feed-out pattern?
df_stst_ts[["onwind_gen_el", "solar_gen_el"]].sum(axis=1).corr(df_stst_ts["h2_feed_in_out"]) # 0.96
df_exp_ts[["onwind_gen_el", "solar_gen_el"]].sum(axis=1).corr(df_exp_ts["h2_feed_in_out"]) # 0.91

In [ ]:
sol_win_gen

In [ ]:
# storage level of 10 largest stores (according to capcity)

fig, axs = plt.subplots(nrows=2, figsize=(14, 8))

for n, ax in zip([stst, exp], axs):
    i_h2_stores_top10 = n.stores.e_nom_opt[i_h2_stores].sort_values(ascending=False).head(10).index
    (n.stores_t.e[i_h2_stores_top10].sum(axis=1) / 1e3 * 3).resample("3h").sum().plot(ax=ax)

fig.tight_layout()

#### Temporal correlation of feed-in pattern

In [ ]:
df_stst_ts.corr()["h2_feed_in"].dropna().sort_values().tail(15)
#df_exp_ts.corr()["h2_feed_in_out"].dropna().sort_values().tail(15)


** Findings:**

feed_in_out:
STST+neg: H2 Electrolysis_con_el (-0.95), BEV charger_con_el (-0.82),
STST+pos: H2 Electrolysis_gen_h2(0.95), BEV charger_gen_el (0.83), solar_gen_el / solar rooftop_gen_el (0.82), battery charger_gen_el (0.79), Sabatier_gen_gas (0.57), offwind-ac_gen_el (0.42)
EXP+neg: H2 Electrolysis_con_el (-0.99), PHS_con_el (-0.63), hydro_gen_el (-0.57)
EXP+pos: H2 Electrolysis_gen_h2 (0.99), BEV charger_gen_el (0.63), solar_gen_el / solar rooftop_gen_el (0.61), onwind_gen_el (0.58), offwind-dc_gen_el (0.50),

feed_in:
STST+neg: H2 Electrolysis_con_el (-0.89), battery discharger_gen_el (-0.75), Sabatier_con_h2 (-0.56), Fischer-Tropsch_con_h2 (-0.24)
STST+pos: solar_gen_el / solar rooftop_gen_el (0.71), battery charger_gen_el (0.65), onwind_gen_el (0.37)
EXP+neg
EXP+pos

feed_out:
STST+neg
STST+pos
EXP+neg
EXP+pos

storage_level:
STST+neg
STST+pos
EXP+neg
EXP+pos

In [ ]:
# Shouldn't there be a high correlation with onwind gen and feed-in-out pattern?
df_stst_ts["h2_feed_in_out"].corr(df_stst_ts["onwind_gen_el"]) # 0.30
df_stst_ts["h2_feed_in_out"].corr(df_stst_ts["solar_gen_el"]) # 0.82

# Primarly h2 from solar generation is feed into the stores, the h2 from onwind generation is rather used directly?

In [ ]:
# no correlation with h2 pipesl
df_exp_ts["h2_feed_in"].corr(df_exp_ts["H2 pipeline_con_h2"]) # 0.82

#### Coming from Storage and Going to Storage

In [ ]:
#### Where ist the hydrogen that is stored coming from and going to?
# this method is bullshit

# exclude
th = 0.01

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    h2_buses = n.stores[n.stores.carrier == "H2"].bus

    # H2 delivering technologies
    n.links[n.links.bus1.isin(h2_buses)]
    i_h2 = n.links[n.links.bus1.isin(h2_buses)].index
    df = pd.DataFrame(n.links_t.p1[i_h2].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_del = abs(df.groupby("carrier").sum())

    result_del_share = result_del / result_del.sum()
    result_del = result_del.loc[result_del_share[result_del_share[0] > th].sort_values(ascending=False, by=0).index]

    # H2 receiving technologies

    # links
    n.links[n.links.bus0.isin(h2_buses)]
    i_h2 = n.links[n.links.bus0.isin(h2_buses)].index
    df = pd.DataFrame(n.links_t.p0[i_h2].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_rec_links = abs(df.groupby("carrier").sum())

    # loads
    i_h2_loads = n.loads[n.loads.bus.isin(h2_buses)].index
    df = pd.DataFrame(n.loads_t.p[i_h2_loads].sum())
    df["carrier"] = df.index.map(n.loads.carrier).values
    result_rec_loads = abs(df.groupby("carrier").sum())

    result_rec = pd.concat([result_rec_links, result_rec_loads])

    result_rec_share = result_rec / result_rec.sum()
    result_rec = result_rec.loc[result_rec_share[result_rec_share[0] > th].sort_values(ascending=False, by=0).index]

    if n==stst:
        ax_del=axs[0, 0]
        ax_rec=axs[0, 1]
        title_del=f"Delivering H2 to storage (STST)"
        title_rec=f"Receiving H2 from storage (STST)"

    elif n==exp:
        ax_del=axs[1, 0]
        ax_rec=axs[1, 1]
        title_del=f"Delivering H2 to storage (EXP)"
        title_rec=f"Receiving H2 from storage (EXP)"

    # plot delivering
    c = [carrier_colors[col] for col in result_del.index]
    percents = result_del.to_numpy() * 100 / result_del.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_del.index, percents)]

    patches, texts = ax_del.pie(result_del.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_del.axis('equal')
    ax_del.set_title(title_del, pad=20, fontweight="bold")

    # plot receiving
    c = [carrier_colors[col] for col in result_rec.index]
    percents = result_rec.to_numpy() * 100 / result_rec.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_rec.index, percents)]

    patches, texts = ax_rec.pie(result_rec.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_rec.axis('equal')
    ax_rec.set_title(title_rec, pad=20, fontweight="bold")

fig.tight_layout(pad=3)
plt.show()

#fig.savefig(f"{PLOT_DIR}/h2_store_del_rev.png")

In [ ]:
res_all = pd.concat([result_rec_links.loc[["Fischer-Tropsch", "H2 Fuel Cell", "H2 liquefaction", "Sabatier"]], result_rec_loads])
res_all / res_all.sum()

In [ ]:
# technologies delivering the hydrogen
n.links[n.links.bus1.isin(h2_buses)].carrier.unique().tolist() + n.generators[n.generators.bus.isin(h2_buses)].carrier.unique().tolist()

In [ ]:
# technologies receiving the hydrogen
n.links[n.links.bus0.isin(h2_buses)].carrier.unique().tolist() + n.loads[n.loads.bus.isin(h2_buses)].carrier.unique().tolist()

In [ ]:
# receiving hydrogen from stores
# sign of pipelines are wrong but does not matter as the amounts are the same

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 11))

for n, ax in zip([stst, exp],axs):

    # data
    h2_buses = n.stores[n.stores.carrier == "H2"].bus
    # links of reciever
    n.links[n.links.bus0.isin(h2_buses)]
    # links indices
    i_h2 = n.links[n.links.bus0.isin(h2_buses)].index
    # production
    df = pd.DataFrame(n.links_t.p0[i_h2].sum())
    # carrier
    df["carrier"] = df.index.map(n.links.carrier).values
    result = abs(df.groupby("carrier").sum())

    c = [carrier_colors[col] for col in result.index]
    percents = result.to_numpy() * 100 / result.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result.index, percents)]

    patches, texts = ax.pie(result.values.flatten(), colors=c, startangle=0, labels=labels)
    ax.axis('equal')
    ax.set_title(f"{n}", pad=20, fontweight="bold")


In [ ]:
n.stores[n.stores.carrier == "Li ion"].bus

In [ ]:
carrier = "Li ion" #"battery" #

# exclude
th = 0.01

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    buses = n.stores[n.stores.carrier == carrier].bus

    # H2 delivering technologies
    i_h2 = n.links[n.links.bus1.isin(buses)].index
    df = pd.DataFrame(n.links_t.p1[i_h2].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_del = abs(df.groupby("carrier").sum())

    result_del_share = result_del / result_del.sum()
    result_del = result_del.loc[result_del_share[result_del_share[0] > th].sort_values(ascending=False, by=0).index]

    # H2 receiving technologies
    # links
    i_h2 = n.links[n.links.bus0.isin(buses)].index
    df = pd.DataFrame(n.links_t.p0[i_h2].sum())
    df["carrier"] = df.index.map(n.links.carrier).values
    result_rec_links = abs(df.groupby("carrier").sum())
    # loads
    i_h2_loads = n.loads[n.loads.bus.isin(buses)].index
    df = pd.DataFrame(n.loads_t.p[i_h2_loads].sum())
    df["carrier"] = df.index.map(n.loads.carrier).values
    result_rec_loads = abs(df.groupby("carrier").sum())

    result_rec = pd.concat([result_rec_links, result_rec_loads])

    result_rec_share = result_rec / result_rec.sum()
    result_rec = result_rec.loc[result_rec_share[result_rec_share[0] > th].sort_values(ascending=False, by=0).index]

    if n==stst:
        ax_del=axs[0, 0]
        ax_rec=axs[0, 1]
        title_del=f"Delivering {carrier} to storage (STST)"
        title_rec=f"Receiving {carrier} from storage (STST)"

    elif n==exp:
        ax_del=axs[1, 0]
        ax_rec=axs[1, 1]
        title_del=f"Delivering {carrier}  to storage (EXP)"
        title_rec=f"Receiving {carrier}  from storage (EXP)"

    # plot delivering
    c = [carrier_colors[col] for col in result_del.index]
    percents = result_del.to_numpy() * 100 / result_del.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_del.index, percents)]

    patches, texts = ax_del.pie(result_del.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_del.axis('equal')
    ax_del.set_title(title_del, pad=20, fontweight="bold")

    # plot receiving
    c = [carrier_colors[col] for col in result_rec.index]
    percents = result_rec.to_numpy() * 100 / result_rec.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(result_rec.index, percents)]

    patches, texts = ax_rec.pie(result_rec.values.flatten(), colors=c, startangle=0, labels=labels)
    ax_rec.axis('equal')
    ax_rec.set_title(title_rec, pad=20, fontweight="bold")

fig.tight_layout(pad=3)
plt.show()

In [ ]:
#### How much of the produced hydrogen is being stored and how much is directly used by consuming technologies?

### Interaction with other sectors

In [ ]:
# Is the methan from Sabatier used to fuel the gas CHP?
# spatial correlation analysis difficualt as all buses are on EU level
df_stst_ons["Sabatier_gen_gas"]#.corr(df_stst_ons["urban central gas CHP_con_gas"])

In [ ]:
# temporal
# also difficult as the gas can be stored (can it?)
df_stst_ts["Sabatier_gen_gas"].corr(df_stst_ts["urban central gas CHP_con_gas"]) # 0.30
df_exp_ts["Sabatier_gen_gas"].corr(df_exp_ts["urban central gas CHP_con_gas"]) # 0.11

In [ ]:
# overall amounts
# what else is gas in the system used for? Or are these losses
df_stst_ts["Sabatier_gen_gas"].sum() / 1e6 # 563
#df_stst_ts["urban central gas CHP_con_gas"].sum() / 1e6 # -340

# where else is gas coming from?
#df_exp_ts["Sabatier_gen_gas"].sum() / 1e6 # 28
df_exp_ts["urban central gas CHP_con_gas"].sum() / 1e6 # -115

## Ideas:
- Calc capacity factors / system load duration curves for electrolysis in different regions and try to investigate in differences (amount of wind generation, solar) Is only excess electrolysis consumed by electrolysis?
- Try to determine connection between electrolysis and excess solar / wind production
- Try to determine percent / amount of excess electricity that is used to produce hydrogen. (Basically all energy from renewables that went into electrolysis?)
- pie chart of percentage of electricity that is used directly, used for Electrolysis, battery, ... (startup script)
- Reproduce graph from Victoria, Zhu et al. 2019 – The role of storage technologies (in hydrogen word document); try to find patterns within the charging of battery and the charging with hydrogen (electrolysis -> fuel cell)
-